# Introduction

This tutorial introduces the basic Auto-PyTorch API together with the classes for featurized and image data.
So far, Auto-PyTorch covers classification and regression on featurized data as well as classification on image data.
For installing Auto-PyTorch, please refer to the github page.

**Disclaimer**: In this notebook, data will be downloaded from the openml project for featurized tasks and CIFAR10 will be downloaded for image classification. Hence, an internet connection is required.

# API

There are classes for featurized tasks (classification, multi-label classification, regression) and image tasks (classification). You can import them via:

In [1]:
from autoPyTorch import (AutoNetClassification,
                         AutoNetMultilabel,
                         AutoNetRegression,
                         AutoNetImageClassification,
                         AutoNetImageClassificationMultipleDatasets)

In [2]:
# Other imports for later usage
import pandas as pd
import numpy as np
import os as os
import openml
import json

Upon initialization of a class, you can specify its configuration. Later, you can override its configuration in each fit call. The *config_preset* allows to constrain the search space to one of *tiny_cs, medium_cs* or *full_cs*. These presets can be seen in *core/presets/*.

In [3]:
autonet = AutoNetClassification(result_logger_dir="logs/")

Here are some useful methods provided by the API:

In [4]:
# Get the current configuration as dict
current_configuration = autonet.get_current_autonet_config()

# Get the ConfigSpace object with all hyperparameters, conditions, default values and default ranges
hyperparameter_search_space = autonet.get_hyperparameter_search_space()

# Print all possible configuration options 
#autonet.print_help()

In [5]:
print(current_configuration)

{'embeddings': ['none'], 'lr_scheduler': ['cosine_annealing', 'plateau'], 'networks': ['shapedresnet'], 'over_sampling_methods': ['smote'], 'preprocessors': ['none', 'truncated_svd', 'power_transformer'], 'target_size_strategies': ['none', 'upsample', 'median'], 'result_logger_dir': 'logs/', 'log_level': 'warning', 'hyperparameter_search_space_updates': None, 'categorical_features': None, 'dataset_name': None, 'run_id': '0', 'task_id': -1, 'algorithm': 'bohb', 'budget_type': 'time', 'eta': 3, 'min_workers': 1, 'working_dir': '.', 'network_interface_name': '{760AB756-6D92-4507-B266-F1952C20D713}', 'memory_limit_mb': 1000000, 'use_tensorboard_logger': False, 'run_worker_on_master_node': True, 'use_pynisher': True, 'validation_split': 0.3, 'refit_validation_split': 0.0, 'cross_validator': 'none', 'cross_validator_args': {}, 'min_budget_for_cv': 0, 'shuffle': True, 'imputation_strategies': ['mean', 'median', 'most_frequent'], 'normalization_strategies': ['none', 'minmax', 'standardize', 'm

In [6]:
print(hyperparameter_search_space)

Configuration space object:
  Hyperparameters:
    CreateDataLoader:batch_size, Type: UniformInteger, Range: [32, 500], Default: 126, on log-scale
    Imputation:strategy, Type: Categorical, Choices: {most_frequent, median, mean}, Default: most_frequent
    InitializationSelector:initialization_method, Type: Categorical, Choices: {sparse, default}, Default: sparse
    InitializationSelector:initializer:initialize_bias, Type: Categorical, Choices: {Yes, No, Zero}, Default: Yes
    InitializationSelector:sparse:sparsity, Type: Constant, Value: 0.9
    LearningrateSchedulerSelector:cosine_annealing:T_max, Type: UniformInteger, Range: [1, 20], Default: 10
    LearningrateSchedulerSelector:cosine_annealing:T_mult, Type: UniformFloat, Range: [1.0, 2.0], Default: 1.5
    LearningrateSchedulerSelector:lr_scheduler, Type: Categorical, Choices: {plateau, cosine_annealing}, Default: plateau
    LearningrateSchedulerSelector:plateau:factor, Type: UniformFloat, Range: [0.05, 0.5], Default: 0.275
  

The most important methods for using Auto-PyTorch are ***fit***, ***refit***, ***score*** and ***predict***.

First, we get some data:

In [7]:
# Get data from the openml task "Supervised Classification on credit-g (https://www.openml.org/t/31)"
task = openml.tasks.get_task(task_id=31)
X, y = task.get_X_and_y()
ind_train, ind_test = task.get_train_test_split_indices()
X_train, Y_train = X[ind_train], y[ind_train]
X_test, Y_test = X[ind_test], y[ind_test]

***fit*** is used to search for a good configuration by fitting configurations chosen by the algorithm (by default BOHB). The incumbent configuration is then returned and stored in the class.

We recommend to have a look at the possible configuration options first. Some of the most important options allow you to set the budget type (epochs or time), run id and task id for cluster usage, tensorboard logging, seed and more.

Here we search for a configuration for 300 seconds with 60-100 s time for fitting each individual configuration.
Use the *validation_split* parameter to specify a split size. You can also pass your own validation set
via *X_val* and *Y_val*. Use *log_level="info"* or *log_level="debug"* for more detailed output.

In [8]:
# Fit (note that the settings are for demonstration, you might need larger budgets)
results_fit = autonet.fit(X_train=X_train,
                          Y_train=Y_train,
                          validation_split=0.3,
                          max_runtime=600,
                          min_budget=30,
                          max_budget=100,
                          refit=True)

# Save fit results as json 
with open("logs/results_fit.json", "w") as file:
    json.dump(results_fit, file)

16:41:33 job (0, 0, 0) failed with exception
Traceback (most recent call last):
  File "C:\Users\monum\Anaconda3\envs\torch_gpu\lib\site-packages\hpbandster\core\worker.py", line 206, in start_computation
    result = {'result': self.compute(*args, config_id=id, **kwargs),
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\autoPyTorch\core\worker.py", line 88, in compute
    result = self.optimize_pipeline(config, config_id, budget, start_time)
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\autoPyTorch\core\worker.py", line 124, in optimize_pipeline
    raise e
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\autoPyTorch\core\worker.py", line 118, in optimize_pipeline
    refit=False, rescore=False, hyperparameter_config_id=config_id, dataset_info=self.dataset_info)
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\autoPyTorch\pipeline\base\pipeline.py", line 60, in fit_pipeline
    return self.root.fit_traverse(**kwargs)
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\a

16:41:33 job (0, 1, 0) failed with exception
Traceback (most recent call last):
  File "C:\Users\monum\Anaconda3\envs\torch_gpu\lib\site-packages\hpbandster\core\worker.py", line 206, in start_computation
    result = {'result': self.compute(*args, config_id=id, **kwargs),
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\autoPyTorch\core\worker.py", line 88, in compute
    result = self.optimize_pipeline(config, config_id, budget, start_time)
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\autoPyTorch\core\worker.py", line 124, in optimize_pipeline
    raise e
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\autoPyTorch\core\worker.py", line 118, in optimize_pipeline
    refit=False, rescore=False, hyperparameter_config_id=config_id, dataset_info=self.dataset_info)
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\autoPyTorch\pipeline\base\pipeline.py", line 60, in fit_pipeline
    return self.root.fit_traverse(**kwargs)
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\a

The Box-Cox transformation can only be applied to strictly positive data
Using yeo-johnson instead


16:41:34 job (2, 0, 0) failed with exception
Traceback (most recent call last):
  File "C:\Users\monum\Anaconda3\envs\torch_gpu\lib\site-packages\hpbandster\core\worker.py", line 206, in start_computation
    result = {'result': self.compute(*args, config_id=id, **kwargs),
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\autoPyTorch\core\worker.py", line 88, in compute
    result = self.optimize_pipeline(config, config_id, budget, start_time)
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\autoPyTorch\core\worker.py", line 124, in optimize_pipeline
    raise e
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\autoPyTorch\core\worker.py", line 118, in optimize_pipeline
    refit=False, rescore=False, hyperparameter_config_id=config_id, dataset_info=self.dataset_info)
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\autoPyTorch\pipeline\base\pipeline.py", line 60, in fit_pipeline
    return self.root.fit_traverse(**kwargs)
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\a

1-th swa update triggered
Scheduler :<class 'autoPyTorch.components.lr_scheduler.lr_schedulers.CosineAnnealingWithRestartsLR'>
Learning rate 0.008778464464573997
2-th swa update triggered
Scheduler :<class 'autoPyTorch.components.lr_scheduler.lr_schedulers.CosineAnnealingWithRestartsLR'>
Learning rate 0.008789580184849703
3-th swa update triggered
Scheduler :<class 'autoPyTorch.components.lr_scheduler.lr_schedulers.CosineAnnealingWithRestartsLR'>
Learning rate 0.00879906009303988
4-th swa update triggered
Scheduler :<class 'autoPyTorch.components.lr_scheduler.lr_schedulers.CosineAnnealingWithRestartsLR'>
Learning rate 0.008807143983985943
5-th swa update triggered
Scheduler :<class 'autoPyTorch.components.lr_scheduler.lr_schedulers.CosineAnnealingWithRestartsLR'>
Learning rate 0.00881403675734239
Wrapping up training!, Final models count:  0
1-th swa update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
Learning rate 0.0005773038319208174
2-th swa update trig

...

2116-th SE update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
2117-th SE update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
2118-th SE update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
2119-th SE update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
2120-th SE update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
2121-th SE update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
2122-th SE update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
2123-th SE update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
2124-th SE update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
2125-th SE update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
2126-th SE update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLR

C:\Users\monum\Anaconda3\envs\torch_gpu\lib\site-packages\numpy\core\_methods.py:195: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
C:\Users\monum\Anaconda3\envs\torch_gpu\lib\site-packages\sklearn\preprocessing\data.py:2863: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
16:48:46 job (7, 0, 0) failed with exception
Traceback (most recent call last):
  File "C:\Users\monum\Anaconda3\envs\torch_gpu\lib\site-packages\hpbandster\core\worker.py", line 206, in start_computation
    result = {'result': self.compute(*args, config_id=id, **kwargs),
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\autoPyTorch\core\worker.py", line 88, in compute
    result = self.optimize_pipeline(config, config_id, budget, start_time)
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\autoPyTorch\core\worker.py", line 124, in optimize_pipeline
    raise e
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\autoPyTorch\c

SWA and SE can't be used simultaneously, prioritizing SE!
lastk:  19


16:50:16 job (7, 0, 1) failed with exception
Traceback (most recent call last):
  File "C:\Users\monum\Anaconda3\envs\torch_gpu\lib\site-packages\hpbandster\core\worker.py", line 206, in start_computation
    result = {'result': self.compute(*args, config_id=id, **kwargs),
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\autoPyTorch\core\worker.py", line 88, in compute
    result = self.optimize_pipeline(config, config_id, budget, start_time)
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\autoPyTorch\core\worker.py", line 124, in optimize_pipeline
    raise e
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\autoPyTorch\core\worker.py", line 118, in optimize_pipeline
    refit=False, rescore=False, hyperparameter_config_id=config_id, dataset_info=self.dataset_info)
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\autoPyTorch\pipeline\base\pipeline.py", line 60, in fit_pipeline
    return self.root.fit_traverse(**kwargs)
  File "C:\Users\monum\Desktop\Project\Auto-PyTorch\a

1-th swa update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
Learning rate 0.0005773038319208174
2-th swa update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
Learning rate 0.0005773038319208174
3-th swa update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
Learning rate 0.0005773038319208174
4-th swa update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
Learning rate 0.0005773038319208174
5-th swa update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
Learning rate 0.0005773038319208174
6-th swa update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
Learning rate 0.0005773038319208174
7-th swa update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
Learning rate 0.0005773038319208174
8-th swa update triggered
Scheduler :<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>
Learning rate 0.000577303831

In [ ]:
model_snapshots = autonet.get_ensemble()

In [ ]:
print(len(model_snapshots))

***refit*** allows you to fit a configuration of your choice for a defined time. By default, the incumbent configuration is refitted during a *fit* call using the *max_budget*. However, *refit* might be useful if you want to fit on the full dataset or even another dataset or if you just want to fit a model without searching.

You can specify a hyperparameter configuration to fit (if you do not specify a configuration the incumbent configuration from the last fit call will be used):

In [ ]:
print(results_fit)

In [ ]:
# See how the random configuration performs (often it just predicts 0)
score = autonet.score(X_test=X_test, Y_test=Y_test)
pred = autonet.predict(X=X_test)

print("Model prediction:", pred[0:10])
print("Accuracy score", score)

In [ ]:
# Refit with sampled hyperparameter config for 120 s. This time on the full dataset.
results_refit = autonet.refit(X_train=X_train,
                              Y_train=Y_train,
                              X_valid=None,
                              Y_valid=None,
                              autonet_config=autonet.get_current_autonet_config(),
                              budget=600)

# Save json
with open("logs/results_refit.json", "w") as file:
    json.dump(results_refit, file)

In [ ]:
# Create an autonet
autonet_config = {
    "result_logger_dir" : "logs/",
    "budget_type" : "epochs",
    "log_level" : "info", 
    "use_tensorboard_logger" : True,
    "validation_split" : 0.0
    }
# Refit with sampled hyperparameter config for 120 s. This time on the full dataset.
results_refit = autonet.refit(X_train=X_train,
                              Y_train=Y_train,
                              X_valid=None,
                              Y_valid=None,
                              autonet_config=autonet.get_current_autonet_config(),
                              budget=30)

# Save json
with open("logs/results_refit.json", "w") as file:
    json.dump(results_refit, file)

***pred*** returns the predictions of the incumbent model. ***score*** can be used to evaluate the model on a test set. 

In [ ]:
# See how the random configuration performs (often it just predicts 0)
score = autonet.score(X_test=X_test, Y_test=Y_test)
pred = autonet.predict(X=X_test)

print("Model prediction:", pred[0:10])
print("Accuracy score", score)

Finally, you can also get the incumbent model as PyTorch Sequential model via

In [ ]:
pytorch_model = autonet.get_pytorch_model()
print(pytorch_model)

# Featurized Data

All classes for featurized data (*AutoNetClassification*, *AutoNetMultilabel*, *AutoNetRegression*) can be used as in the example above. The only difference is the type of labels they accept.

# Image Data

Auto-PyTorch provides two classes for image data. *autonet_image_classification* can be used for classification for images. The *autonet_multi_image_classification* class allows to search for configurations for image classification across multiple datasets. This means Auto-PyTorch will try to choose a configuration that works well on all given datasets.

In [ ]:
# Load classes
autonet_image_classification = AutoNetImageClassification(config_preset="full_cs", result_logger_dir="logs/")
autonet_multi_image_classification = AutoNetImageClassificationMultipleDatasets(config_preset="tiny_cs", result_logger_dir="logs/")

For passing your image data, you have two options (note that arrays are expected):

I) Via a path to a comma-separated value file, which in turn contains the paths to the images and the image labels (note header is assumed to be None):

In [ ]:
csv_dir = os.path.abspath("../../datasets/example.csv")

X_train = np.array([csv_dir])
Y_train = np.array([0])

II) directly passing the paths to the images and the labels

In [ ]:
df = pd.read_csv(csv_dir, header=None)
X_train = df.values[:,0]
Y_train = df.values[:,1]

Make sure you specify *image_root_folders* if the paths to the images are not specified from your current working directory. You can also specify *images_shape* to up- or downscale images.

Using the flag *save_checkpoints=True* will save checkpoints to the result directory:

In [ ]:
autonet_image_classification.fit(X_train=X_train,
                                 Y_train=Y_train,
                                 images_shape=[3,32,32],
                                 min_budget=200,
                                 max_budget=400,
                                 max_runtime=600,
                                 save_checkpoints=True,
                                 images_root_folders=[os.path.abspath("../../datasets/example_images")])

Auto-PyTorch also supports some common datasets. By passing a comma-separated value file with just one line, e.g. "CIFAR10, 0" and specifying *default_dataset_download_dir* it will automatically download the data and use it for searching. Supported datasets are CIFAR10, CIFAR100, SVHN and MNIST.

In [ ]:
path_to_cifar_csv = os.path.abspath("../../datasets/CIFAR10.csv")

autonet_image_classification.fit(X_train=np.array([path_to_cifar_csv]),
                                 Y_train=np.array([0]),
                                 min_budget=600,
                                 max_budget=900,
                                 max_runtime=1800,
                                 default_dataset_download_dir="./datasets",
                                 images_root_folders=["./datasets"])

For searching across multiple datasets, pass multiple csv files to the corresponding Auto-PyTorch class. Make sure your specify *images_root_folders* for each of them.

In [ ]:
autonet_multi_image_classification.fit(X_train=np.array([path_to_cifar_csv, csv_dir]),
                                       Y_train=np.array([0]),
                                       min_budget=1500,
                                       max_budget=2000,
                                       max_runtime=4000,
                                       default_dataset_download_dir="./datasets",
                                       images_root_folders=["./datasets", "./datasets/example_images"])